### Data Preparation

In [ ]:
def build_k_indices(y, k_fold, seed):
    '''
    Build k randomly selected disjoint lists of indices to be used in 
    cross-validation. 
    '''
    
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    
    # Create a random permutation of all numbers from 1 to num_row
    indices = np.random.permutation(num_row)
    
    # Select k partitions of the random permutation
    k_indices = [indices[k * interval: (k + 1) * interval]
                    for k in range(k_fold)]
    
    return np.array(k_indices)

def cross_data(y, x, k_indices, k):
    '''
    Given a feature set, label set, and k disjoint lists of indices, 
    return the k-th partition (training + test for both features and labels). 
    '''

    # get k'th subgroup in test, others in train:
    x_tr = x.iloc[k_indices[np.arange(len(k_indices)) != k].flatten()]
    x_te = x.iloc[k_indices[k]]

    # Same for y
    y_tr = y.iloc[k_indices[np.arange(len(k_indices)) != k].flatten()]
    y_te = y.iloc[k_indices[k]]

    return x_tr, x_te, y_tr, y_te

### Feature Selection

In [ ]:
def greedy_backward_selection(y, x, k_fold, decision_boundary, metric):
    '''
    Perform greedy backward selection: until there are only one feature left,
    try to remove each one of the remaining features and remove the one that 
    has the lowest score.
    (Uses cross-validation with k_fold partitions).
    '''
    
    selected_features = { x.shape[1] : (x, evaluate(y, x, k_fold, decision_boundary, metric, LogisticRegression())) }
    
    x_iter = x
    
    while len(x_iter.columns) > 1:
        
        best_x = None
        best_result = -float('inf')
        
        for column in x_iter.columns:
        
            x_trim = x_iter.drop(column, axis=1)
            
            result = evaluate(y, x_trim, k_fold, decision_boundary, metric, LogisticRegression())
            
            if result > best_result:
                
                best_x = x_trim
                best_result = result
        
        x_iter = best_x
        selected_features[best_x.shape[1]] = (best_x, best_result)
    
    return selected_features
    
    
def evaluate(y, x, k_fold, decision_boundary, metric, classifier_model):
    '''
    Calculate a given metric using cross-validation over k_fold partitions.
    '''
    
    result = 0
    
    k_indices = build_k_indices(y, k_fold, 0)
    
    for k in range(0, k_fold):
        
        # Get the k-th split data set
        x_tr, x_te, y_tr, y_te = cross_data(y, x, k_indices, k)
        
        # Train a classifier on the training data
        classifier = classifier_model.fit(x_tr, y_tr)
        
        # Predict labels
        y_pred = classify(classifier, x_te, decision_boundary)
    
        # Compute confusion matrix
        result += metric(y_te, y_pred) / k_fold
        
    return result

def accuracy_metric(y_te, y_pred):
    '''
    Accuracy of the classified labels.
    '''

    return accuracy(confusion_matrix(y_te, y_pred))

### Analysis of Results

- $\displaystyle\text{Precision} = \frac{\text{TP}}{\text{TP}\;+\;\text{FP}}$


- $\displaystyle\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$


- $\displaystyle\text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$


- $\displaystyle\text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$

In [ ]:
def confusion_matrix(y_te, y_pred):
    return [
        [cm_item(y_te, y_pred, actual=1, pred= 1), cm_item(y_te, y_pred, actual=-1, pred= 1)],
        [cm_item(y_te, y_pred, actual=1, pred=-1), cm_item(y_te, y_pred, actual=-1, pred=-1)]
    ]

tp = lambda m: m[0][0]
fp = lambda m: m[0][1]
fn = lambda m: m[1][0]
tn = lambda m: m[1][1]

In [ ]:
def precision(m, pos=True):
    if pos:
        return tp(m) / (tp(m) + fp(m))
    else:
        return tn(m) / (tn(m) + fn(m))

def recall(m, pos=True):
    if pos:
        return tp(m) / (tp(m) + fn(m))
    else:
        return tn(m) / (tn(m) + fp(m))

def accuracy(m, pos=True):
    return (tp(m) + tn(m)) / np.sum(m)

def f1_score(m, pos=True):
    return 2 * (precision(m, pos) * recall(m, pos)) / (precision(m, pos) + recall(m, pos))